Este script sirve para: <br>
1.- Abrir el navegador<br>
2.- Ir a RENATI (https://renati.sunedu.gob.pe/)<br>
3.- Lanzar búsquedas usando un términos específicos<br>
4.- Extraer los urls de todos los resultados<br>
5.- Guardar los urls en un archivo externo.<br>
Dichos urls seran datos de entrada para obtener la informacion detallada de cada item (tesis, articulo, etc.)

# 1. Librerías

In [1]:
import pandas as pd
import time, os, re

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# 2. Variables globales

In [3]:
RENATI_url = 'https://renati.sunedu.gob.pe'

In [4]:
#esta línea debe actualizarse si se cambia de versión de chromedriver
chrome_path = r'.\ext\111.0.5563.64\chromedriver.exe'

# 3. Definición de funciones (I)

In [5]:
#Función para iniciar una nueva instancia del navegador Chrome
def openWebNav():
    
    WebNavObj = webdriver.Chrome(executable_path = chrome_path)
    
    return WebNavObj

In [6]:
def getCurrentHTML(WebNavObj):
    ps = WebNavObj.page_source
    soupObj =BeautifulSoup(ps, 'lxml')
    
    return soupObj

In [7]:
def runRENATIsearch(WebNavObj, kwStr):
    selStr = "input[type='text']"

    try:
        sBox = WebDriverWait(WebNavObj,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, selStr)))
        sBox.clear()
        sBox.send_keys(kwStr + '\n')
    except:
        print(f"\tError al buscar {kwStr}. Hazlo manualmente...")
        input("\tEn espera...(presione tecla)...")

In [8]:
def getNrRENATI(WebNavObj):
    
    soupObj = getCurrentHTML(WebNavObj)
    tagNr = ['div','alert alert-info']

    NrElem = soupObj.find(tagNr[0],{'class':tagNr[1]})
    
    if NrElem is not None:
        try:
            Nr = NrElem.text
            Nr = Nr.split()
            Nr = Nr[3].replace(".","")
            Nr = int(Nr)
        except:
            print("No se ubicó la cantidad de resultados.")
            Nr = input("Ingrese el número manualmente =>")
            Nr = int(Nr)
    else:
        Nr = 0
        
    return Nr    

In [9]:
def getRENATIurls(soupObj):
    #Encontrar todos los elementos 'a' con href que comiencen con '/handle/sunedu/'
    RENATIurls = soupObj.find_all('a', href=lambda href: href and href.startswith('/handle/sunedu/'))
    
    href_values = []
    
    #Iterar sobre RENATIurls y extraer el valor de href
    for e in RENATIurls:
        href = e.get('href')
        href_values.append(href)
        
    return href_values  

In [10]:
def saveRENATIurlsList(LstObj, kwStr, pathStr):
    
    fName = "RENATI-" + kwStr + " (" + str(len(LstObj)) + ")" + ".txt"
    fPath = pathStr + fName
    
    with open(fPath, "w") as file:
        for e in LstObj:
            file.write(str(e) + "\n")

# 4. Programa principal para obtener lista de URLs de RENATI

## 4.1. Abrir navegador

In [14]:
WebNavObj = openWebNav()
WebNavObj.maximize_window()
WebNavObj.get(RENATI_url)

## 4.2. Lanzando una busqueda

In [15]:
kwList = ["aerogenerador", "energía eólica", "eólica", "generador eólico", "turbina eólica", "wind energy", "wind turbine"]

In [16]:
for kwStr in kwList:
    
    #Lanzando búsqueda
    runRENATIsearch(WebNavObj, kwStr)
    
    #Obteniendo el número de resultados
    Nr = getNrRENATI(WebNavObj)
    print(f"\tLa búsqueda del término {kwStr} arrojó {Nr} documentos.")
    
    nTabs = int(Nr/10)
    
    RENATI_URLS = []
    
    for t in range(1, nTabs + 2, 1):
        soupObj = getCurrentHTML(WebNavObj) #Código HTML de la página actual        
        tmpURLs = getRENATIurls(soupObj)    #URLs de la página actual
        RENATI_URLS.extend(tmpURLs)         #Guardando lista de URLS en una lista acumulada
        
        #Visitando la siguiente pestaña
        time.sleep(1)
        try:
            sigBut = WebNavObj.find_element_by_link_text("Siguiente")
            sigBut.click()
        except:
            print("\t\tEl botón 'Siguiente' esta deshabilitado.")
    #Guardando la lista de enlaces
    RENATI_URLS = list(set(RENATI_URLS))
    saveRENATIurlsList(RENATI_URLS, kwStr, './lists/RENATI/')

	La búsqueda del término aerogenerador arrojó 169 documentos.
		El botón 'Siguiente' esta deshabilitado.
	La búsqueda del término energía eólica arrojó 329 documentos.
		El botón 'Siguiente' esta deshabilitado.
	La búsqueda del término eólica arrojó 384 documentos.
		El botón 'Siguiente' esta deshabilitado.
	La búsqueda del término generador eólico arrojó 55 documentos.
		El botón 'Siguiente' esta deshabilitado.
	La búsqueda del término turbina eólica arrojó 72 documentos.
		El botón 'Siguiente' esta deshabilitado.
	La búsqueda del término wind energy arrojó 177 documentos.
		El botón 'Siguiente' esta deshabilitado.
	La búsqueda del término wind turbine arrojó 44 documentos.
		El botón 'Siguiente' esta deshabilitado.


## 4.3. Cerrar navegador

In [17]:
WebNavObj.close()
WebNavObj.quit()

#--> ====================================================================================

In [19]:
input("ATENCIÓN!!  Leer mensaje...")

ATENCIÓN!!  Leer mensaje... 


' '

No ejecutar el código lineas abajo si la carpeta '.\lists\RENATI' esta vacía.

# 5. Programa principal para obtener detalles a partir de listado de URLs de RENATI

# 5.1. Definición de funciones (II)

In [20]:
def consolidarRENATItxt(pathStr):
    files = [f for f in os.listdir(pathStr) if re.match(r'RENATI.*\.txt', f)]
    
    RENATIurls = set()
    
    for txtf in files:
        with open(os.path.join(pathStr, txtf), 'r') as f:
            for linea in f:
                RENATIurls.add(linea.strip())
                
    return list(RENATIurls)

In [21]:
def getBasicMtdata(basicBlock):
    metadata_dict = {}

    labels = basicBlock.find_all(class_='metadataFieldLabel')
    values = basicBlock.find_all(class_='metadataFieldValue')

    for i in range(len(labels)):
        metadata_dict[labels[i].text.strip()] = values[i].text.strip()
    return metadata_dict

In [22]:
def getPageMtdata(soupObj, urlStr):
    mtdataArr = soupObj.find_all(class_ = 'metadataField')
    
    pageMtdata = {'URL:':urlStr}
    for i in range(len(mtdataArr)):
        dict_temp = getBasicMtdata(mtdataArr[i])
        pageMtdata.update(dict_temp)
    return pageMtdata

In [23]:
def dictArrtoDf(dictArrObj):
    # obtener lista de claves únicas
    headers = list(set().union(*dictArrObj))
    
    # crear DataFrame con claves como columnas
    df = pd.DataFrame(columns = headers)
    
    # agregar cada diccionario como fila en el DataFrame
    for dic in dictArrObj:
        # Se convierte el diccionario en un DataFrame
        df_temp = pd.DataFrame.from_dict(dic, orient='index').T
        
        # Se concatena el DataFrame temporal con el DataFrame acumulado
        df = pd.concat([df, df_temp], ignore_index=True, sort=False)
    
    return df  

## 5.1. Cargando datos de los enlaces

In [24]:
RENATIurls = consolidarRENATItxt('./lists/RENATI/')
print(f'Se cargaron un total de {len(RENATIurls)} enlaces para analizar.')

Se cargaron un total de 571 enlaces para analizar.


## 5.1. Abrir navegador

In [25]:
WebNavObj = openWebNav()
WebNavObj.maximize_window()
WebNavObj.get(RENATI_url)

Esperar que cargue la pagina, puede demorar un poco

## 5.2. Visitando páginas y recolectando información generando un arreglo de diccionarios

In [26]:
RENATIMtdata = []
for i in range(len(RENATIurls)): #
    WebNavObj.get(RENATI_url + RENATIurls[i])
    time.sleep(0.5)
    
    soupObj = getCurrentHTML(WebNavObj)
    urlMtdata = getPageMtdata(soupObj, RENATI_url + RENATIurls[i])
    
    RENATIMtdata.append(urlMtdata)

La ejecución de esta celda puede demorar dependiendo de cuantos registros tenga que visitar.

## 5.3. Generando un dataframe a partir del arreglo de diccionarios

In [27]:
RENATIrecords = dictArrtoDf(RENATIMtdata)

In [28]:
RENATIrecords.to_excel('RENATI_RawData.xlsx', index = False)

### 5.3.1. Extrayendo un dataframe solo con las columnas mas relevantes

In [29]:
ColRed = ['URL:',
     'Enlace al repositorio:',
     'Aparece en las colecciones:',
     'Institución:',
     'Institución que otorga el grado o título:',
     'Disciplina académico-profesional:',
     'Grado o título:',
     'Título:',
     'Autor(es):',
     'Asesor(es):',
     'Fecha de publicación:',
     'Palabras clave:']

In [30]:
RENATIrecordsRed = RENATIrecords.loc[:, ColRed]

In [31]:
RENATIrecordsRed.to_excel('RENATI_reduced.xlsx', index = False)

# 6. Programa principal para normalizar titulo y autores y generar un identificador único para detectar registros dusplicados

In [32]:
import hashlib

## 6.1. Definición de funciones

In [33]:
def normalize_title(title):
    # Eliminar signos de puntuación y espacios en blanco extra
    normalized_title = title.strip().replace(' ','').replace('"','').replace('.', '').replace(',', '').replace('-', '').lower()    
    
    return normalized_title

In [34]:
def normalize_authors(authors):
    # Eliminar signos de puntuación y convertir todo a minúsculas
    normalized_authors = authors.lower().replace(',', '').replace('.', '').replace(' ','').replace('-', '')
    
    # Dividir los nombres de los autores y ordenarlos alfabéticamente
    author_list = sorted(normalized_authors.split(';'))
    
    # Unir los nombres de los autores separados por un punto y coma
    normalized_authors = ';'.join(author_list)
    return normalized_authors

In [35]:
def generate_unique_id(title, authors):
    # Normalizar la columna TITULO y la columna AUTORES
    normalized_title = normalize_title(title)
    normalized_authors = normalize_authors(authors)
    
    # Generar un código único solo para el título
    title_id = hashlib.md5(normalized_title.encode()).hexdigest()
    
    # Generar un código único solo para los autores
    authors_id = hashlib.md5(normalized_authors.encode()).hexdigest()
    
    # Concatenar la columna TITULO y la columna AUTORES en una sola cadena
    combined_string = normalized_title + '|' + normalized_authors
    
    # Generar un código único para el registro completo
    unique_id = hashlib.md5(combined_string.encode()).hexdigest()
    return title_id, authors_id, unique_id

## 6.2. Generando identificadores únicos para el titulo, autores y combinación de ambos

In [36]:
#-->
#RENATIrecordsRed = pd.read_excel(r'.\_results\Book1.xlsx')

In [37]:
RENATIrecordsRed['UID_Tit'], RENATIrecordsRed['UID_Aut'], RENATIrecordsRed['UID'] = zip(*RENATIrecordsRed.apply(lambda row: generate_unique_id(row['Título:'], row['Autor(es):']), axis=1))

In [38]:
RENATIrecordsRed.head()

,URL:,Enlace al repositorio:,Aparece en las colecciones:,Institución:,Institución que otorga el grado o título:,Disciplina académico-profesional:,Grado o título:,Título:,Autor(es):,Asesor(es):,Fecha de publicación:,Palabras clave:,UID_Tit,UID_Aut,UID
0,https://renati.sunedu.gob.pe/handle/sunedu/300...,https://hdl.handle.net/11537/21870,Título profesional - Tesis,Universidad Privada del Norte,Universidad Privada del Norte. Facultad de Ing...,Ingeniería Civil,Bachiller en Ingeniería Civil,Estabilización de suelos con erosión eólica pa...,"Sánchez Coronado, Marcial Alexander; Romero Pu...","Díaz García, Gonzalo Hugo",13-may-2019,Erosión; Estudios de suelos; Aguas residuales ...,bd009486bf1906fcfcf58e0a8f219007,50cff5acadcf7e7cc4968f6ab498e309,1171ff6f615f2ecf4f4a6c2406b27979
1,https://renati.sunedu.gob.pe/handle/sunedu/266...,http://hdl.handle.net/20.500.12404/6095,Título profesional - Tesis,Pontificia Universidad Católica del Perú,Pontificia Universidad Católica del Perú. Facu...,Ingeniería Electrónica,Ingeniero Electrónico,Diseño e implementación de un equipo de metrol...,"Villanueva Blas, Lis Mariela","Melgarejo Ponte, Óscar Antonio",23-jun-2015,Energía eólica--Aparatos e instrumentos.; Sist...,b978e1b0e9ba21d9f828e15495871381,23e9305cc74d5f24b21b34b299a8480d,f81e77e62f2f288dcf44c6982ee84f85
2,https://renati.sunedu.gob.pe/handle/sunedu/301...,https://hdl.handle.net/11537/10088,Título profesional - Tesis,Universidad Privada del Norte,Universidad Privada del Norte. Facultad de Neg...,Administración y Servicios Turísticos,Licenciado en Administración y Servicios Turís...,Condiciones turísticas del turismo esotérico e...,"Medina Zavaleta, Clary Sandy; Núñez Salazar, E...","Hurtado Castañeda, Jamy",2-jul-2016,Desarrollo sostenible; Turismo; Fomento del tu...,21526a051ab13e7406b3861f70cc646d,27e0452405cadacbecca016b47013d88,636ad7921a3d1cf3db39eb5e9a5e7073
3,https://renati.sunedu.gob.pe/handle/sunedu/299...,http://dspace.unitru.edu.pe/handle/UNITRU/12814,Doctorado - Tesis,Universidad Nacional de Trujillo,Universidad Nacional de Trujillo.Escuela de Po...,Doctorado en Ciencias e Ingeniería,Doctor en Ciencias e Ingeniería,Evaluación de un modelo para las decisiones de...,"Hurtado Zamora, Oswaldo","Benites Gutiérrez, Luis Alberto",2017,Modelo para las decisiones de inversión energí...,7db3624d0e25fdae49e1b5940d6fb010,964e90d6d8601b19015b0319349bf840,f8630394c56fd7f580a7bccc652d98d1
4,https://renati.sunedu.gob.pe/handle/sunedu/294...,https://hdl.handle.net/20.500.12692/35910,Título profesional - Tesis,Universidad César Vallejo,Universidad César Vallejo. Facultad de Ingenie...,Ingeniería Mecánica Eléctrica,Ingeniero Mecánico Electricista,Estudio de viabilidad de lanchas solares para ...,"Oblitas Guevara, Nelbar","Díaz Rubio, Deciderio Enrique",2019,Energía; Viabilidad; Fotovoltaico; Motor fuera...,2b18ace22190f1157cd58ec245fa8849,3dc0217b471b79844dede332d952de12,4992597427166937080827bf262e00e1


In [39]:
RENATIrecordsRed.to_excel('RENATI_reduced_UID.xlsx', index = False)

## 6.3. Cerrar navegador

In [40]:
WebNavObj.close()
WebNavObj.quit()